In [ ]:

import pandas as pd
from datasets import Dataset
#read data
data = pd.read_excel(r"C:\Users\me513\Downloads\MAQA_30_words_Classifier_train_sorted.xlsx")
grouped_data = data.groupby('category')
group_key ='امراض الدم'
squad = grouped_data.get_group(group_key)
print(squad.count())
squad['question']=squad['q_body']
squad['answers']=squad['a_body']
# Step 2: Convert data to Hugging Face Dataset format
dataset = Dataset.from_pandas(squad)
# Drop the column(s) you want
columns_to_drop = ['q_body', 'a_body','category','q_body_count','a_body_count']
dataset = dataset.remove_columns(columns_to_drop)

In [ ]:
del(data)
del(squad)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name ='abdalrahmanshahrour/AraBART-summ'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples['question'], padding='max_length', truncation=True, max_length=30)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['answers'], padding='max_length', truncation=True, max_length=30)
        
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results2',          # output directory
    num_train_epochs=50,              # total number of training epochs
    per_gpu_train_batch_size=64,   # batch size for training
    per_gpu_eval_batch_size=64,    # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=5,
    report_to="none", # disable Wandb reporting
    use_cpu=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    #eval_dataset=tokenized_val_dataset,
)

trainer.train()


In [ ]:
import torch
torch.device('cuda')

In [ ]:
import os

# Define the output directory
output_dir = r"C:\Users\me513\Downloads\balance\50امراض الدم"

# Create the directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Save the model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
